In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import deque, defaultdict
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [2]:
input_text = au.read_txt_file_lines()

In [3]:
designs = input_text[0].split(', ')
patterns = input_text[2:]

In [14]:
## This algorithm works, but is too slow:
possible = 0 

for p_start in patterns:

    queue = deque([p_start])
    
    while len(queue) > 0:
        p = queue.popleft()

        for d in designs:
            if p[:len(d)] == d:
                if len(p) == len(d):
                    possible += 1
                    queue = []
                    break 

                queue.append(p[len(d):])

In [22]:
## inspecting data: all single letters are available except u(!!)

dict_designs = defaultdict(list)
for d in designs:
    dict_designs[len(d)].append(d)

print(dict_designs[1])
print(dict_designs[2])

['r', 'g', 'w', 'b']
['ug', 'wb', 'bw', 'rr', 'br', 'ru', 'gb', 'rg', 'gw', 'bu', 'rb', 'wu', 'ww', 'wg', 'ur', 'uw', 'bg', 'ub', 'rw', 'gr', 'gg', 'bb', 'wr']


Can we alter the algorithm to tailor towards `u`?
- We only need to find fancy patterns to match the `u`'s
- Idea: find the first `u`, match it to all designs (with `u`), if successful then just need consider what comes after.
- Runtime: 2mins. Note: could be (a lot?) faster by grouping designs by len and if 1 works skipping all others.

In [30]:
tmp = [d for d in designs if 'u' in d]
designs_u = []
for d in tmp:
    for i_s, s in enumerate(d):
        if s == 'u':  # first occurence
            designs_u.append((d, i_s))
            break

possible = 0

for p_start in tqdm(patterns):
    if 'u' not in p_start:
        possible += 1
        continue

    queue = deque([p_start])
    
    while len(queue) > 0:
        p = queue.popleft()
        if 'u' not in p:
            possible += 1
            queue = []
            break 

        for i_s, s in enumerate(p):  # find ind of first u
            if s == 'u':  # first occurrence:
                break 

        for (d, i_u) in designs_u:
            if i_u > i_s:  # d u occurs later than p u 
                continue 

            if p[i_s - i_u:i_s - i_u + len(d)] == d:
                if 'u' not in p[i_s - i_u + len(d):]:
                    possible += 1
                    queue = []
                    break 
                queue.append(p[i_s - i_u + len(d):])



100%|██████████| 400/400 [01:55<00:00,  3.46it/s]


In [31]:
possible

333

## part 2 
- essentially just remove break clause of original algorithm.. But would take ages to find all perms. 
- Can use part 1 to at least filter 50% that is impossible. 
- Now also need to do the non-`u` sequences

In [32]:
len(designs)

447

In [7]:
max_len_d = max([len(x) for x in designs])

dict_designs_by_len = {x: [d for d in designs if len(d) <= x] for x in range(1, max_len_d + 1)}

def get_n_perms(s):
    if len(s) == 0:
        return 1

    tmp_designs = dict_designs_by_len[min(len(s), max_len_d)]

    for d in tmp_designs:
        if s[:len(d)] == d:
            n = get_n_perms(s[len(d):]) + 1

    return 0


In [8]:
designs[:4]

['ggbwrgwu', 'bbbgru', 'wwbg', 'gbuuruwu']

In [10]:
for x in itertools.permutations(designs[:4], 2):
    print(x)

('ggbwrgwu', 'bbbgru')
('ggbwrgwu', 'wwbg')
('ggbwrgwu', 'gbuuruwu')
('bbbgru', 'ggbwrgwu')
('bbbgru', 'wwbg')
('bbbgru', 'gbuuruwu')
('wwbg', 'ggbwrgwu')
('wwbg', 'bbbgru')
('wwbg', 'gbuuruwu')
('gbuuruwu', 'ggbwrgwu')
('gbuuruwu', 'bbbgru')
('gbuuruwu', 'wwbg')


In [33]:
import functools 
@functools.cache
def permute(d):
    res = []
    if len(d) == 1:
        res = [d[0]]
    else:
        for pos, ch in enumerate(d):
            for perm in permute(tuple(list(d[:pos]) + list(d[pos + 1:]))):
                res += [ch + perm]
    return res

designs.sort(key=len)

In [1]:
permute(tuple(designs[:10]))

NameError: name 'permute' is not defined

In [30]:
(0, 1, 2)[:1]

(0,)